# Tuberculosis Diagnosis using Transfer Learning
This notebook is written and executed by **Dr Raheel Siddiqi** on *03-10-2019*. The notebook presents an experiment to classify X-ray images as 'NORMAL' or 'containing manifestation of Tuberculosis (TB)' i.e. it is a binary classification problem. Transfer Learning is used to exploit the feature extractor of the *VGG16* pre-trained model. The dataset used is: [**China Set - The Shenzhen set - Chest X-ray Database**](https://www.kaggle.com/kmader/pulmonary-chest-xray-abnormalities). 5-fold cross validation is used to evaluate the model.

In [1]:
import tensorflow as tf
from tensorflow.python import keras

print('Tensorflow Version: ', tf.__version__)
print('Keras Version: ', keras.__version__)

Tensorflow Version:  1.13.1
Keras Version:  2.2.4-tf


In [2]:
from tensorflow.python.keras.applications import VGG16
import os
import numpy as np
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import models
from tensorflow.python.keras import layers
from tensorflow.python.keras import optimizers

def get_model():
    model = models.Sequential()
    conv_base=VGG16(weights='imagenet',include_top=False,input_shape=(100,100,3))
    model.add(conv_base)
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer=optimizers.Adam(lr=1e-4),metrics=['accuracy'])
    return model   

In [3]:
image_height = 100
image_width = 100
batch_size = 2
no_of_epochs  = 15

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
%matplotlib inline

DATADIR = "D:\\TensorFlow Programs\\TB Diagnosis\\ChinaSet_AllFiles\\ChinaSet_AllFiles\\ChinaSetImages"

CATEGORIES = ["NORMAL", "TB"]

data=[]

for category in CATEGORIES:  
    path = os.path.join(DATADIR,category)
    class_num=CATEGORIES.index(category)
    for img in tqdm(os.listdir(path)):  
        try:
            img_array = cv2.imread(os.path.join(path,img))
            img_array = cv2.resize(img_array, (image_height, image_width))
            img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)        
            img_array = img_array.astype(np.float32)/255.
            data.append([img_array, class_num])
        except Exception as e:   
            pass

print(len(data))

100%|████████████████████████████████████████████████████████████████████████████████| 336/336 [00:34<00:00,  7.82it/s]


662


In [5]:
import random

random.shuffle(data)
for sample in data[:10]:
    print(sample[1])

0
1
0
1
1
1
0
1
1
0


In [6]:
X = []
y = []

for features,label in data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, image_width, image_height, 3)
print(X.shape)

(662, 100, 100, 3)


In [7]:
print(len(X))

662


In [8]:
print(X.shape)

(662, 100, 100, 3)


In [9]:
k=5
num_validation_samples=len(X)//k
validation_scores=[]
for fold in range(k):
    validation_data=X[num_validation_samples*fold:num_validation_samples*(fold+1)]
    validation_labels=y[num_validation_samples*fold:num_validation_samples*(fold+1)]
    if fold==0:
        training_data=X[num_validation_samples*(fold+1):]
        training_labels=y[num_validation_samples*(fold+1):]    
    else:
        training_data=np.append(X[:num_validation_samples*fold], X[num_validation_samples*(fold+1):],axis=0)
        training_labels=np.append(y[:num_validation_samples*fold], y[num_validation_samples*(fold+1):],axis=0)
    model=get_model()
    model.fit(training_data,training_labels,batch_size=4,epochs=5)
    validation_score=model.evaluate(validation_data,validation_labels)
    validation_scores.append(validation_score[1])
print('Average Validation Score: ', np.average(validation_scores))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/5
530/530 [==============================] - ETA: 8:21 - loss: 0.5981 - acc: 0.750 - ETA: 4:16 - loss: 0.7737 - acc: 0.625 - ETA: 2:54 - loss: 0.7652 - acc: 0.500 - ETA: 2:14 - loss: 0.6827 - acc: 0.625 - ETA: 1:49 - loss: 0.5803 - acc: 0.700 - ETA: 1:32 - loss: 0.7516 - acc: 0.666 - ETA: 1:20 - loss: 0.8662 - acc: 0.607 - ETA: 1:12 - loss: 0.8270 - acc: 0.625 - ETA: 1:05 - loss: 0.8046 - acc: 0.638 - ETA: 59s - loss: 0.7783 - acc: 0.650 - ETA: 55s - loss: 0.7912 - acc: 0.59 - ETA: 51s - loss: 0.7781 - acc: 0.60 - ETA: 48s - loss: 0.8182 - acc: 0.59 - ETA: 45s - loss: 0.8163 - acc: 0.58 - ETA: 42s - loss: 0.8210 - acc: 0.55 - ETA: 40s - loss: 0.8090 - acc: 0.57 - ETA: 38s - loss: 0.7953 - acc: 0.58 - ETA: 37s - loss: 0.7941 - acc: 0.58 - ETA: 35s - loss: 0.7921 - acc: 0.57 - ETA: 34s - loss: 0.7893 - acc: 0.57 - ETA: 32s - loss: 0.7856 - acc: 0.57 - ETA: 31s -

530/530 [==============================] - ETA: 15s - loss: 0.6655 - acc: 0.50 - ETA: 15s - loss: 0.6598 - acc: 0.62 - ETA: 15s - loss: 0.6717 - acc: 0.50 - ETA: 15s - loss: 0.6805 - acc: 0.43 - ETA: 15s - loss: 0.6763 - acc: 0.50 - ETA: 14s - loss: 0.6755 - acc: 0.54 - ETA: 14s - loss: 0.6709 - acc: 0.57 - ETA: 14s - loss: 0.6678 - acc: 0.59 - ETA: 14s - loss: 0.6689 - acc: 0.58 - ETA: 14s - loss: 0.6645 - acc: 0.60 - ETA: 14s - loss: 0.6677 - acc: 0.59 - ETA: 14s - loss: 0.6715 - acc: 0.56 - ETA: 14s - loss: 0.6691 - acc: 0.55 - ETA: 14s - loss: 0.6644 - acc: 0.58 - ETA: 13s - loss: 0.6606 - acc: 0.61 - ETA: 13s - loss: 0.6643 - acc: 0.59 - ETA: 13s - loss: 0.6679 - acc: 0.57 - ETA: 13s - loss: 0.6730 - acc: 0.54 - ETA: 13s - loss: 0.6741 - acc: 0.55 - ETA: 13s - loss: 0.6753 - acc: 0.55 - ETA: 13s - loss: 0.6748 - acc: 0.55 - ETA: 13s - loss: 0.6757 - acc: 0.55 - ETA: 12s - loss: 0.6773 - acc: 0.53 - ETA: 12s - loss: 0.6782 - acc: 0.53 - ETA: 12s - loss: 0.6793 - acc: 0.51 - ETA: 12

530/530 [==============================] - ETA: 14s - loss: 0.6932 - acc: 0.50 - ETA: 14s - loss: 0.6922 - acc: 0.62 - ETA: 14s - loss: 0.6925 - acc: 0.58 - ETA: 15s - loss: 0.6922 - acc: 0.62 - ETA: 14s - loss: 0.6924 - acc: 0.60 - ETA: 14s - loss: 0.6925 - acc: 0.58 - ETA: 14s - loss: 0.6923 - acc: 0.60 - ETA: 14s - loss: 0.6927 - acc: 0.56 - ETA: 14s - loss: 0.6927 - acc: 0.55 - ETA: 14s - loss: 0.6926 - acc: 0.60 - ETA: 14s - loss: 0.6927 - acc: 0.59 - ETA: 14s - loss: 0.6927 - acc: 0.58 - ETA: 14s - loss: 0.6929 - acc: 0.55 - ETA: 13s - loss: 0.6933 - acc: 0.51 - ETA: 13s - loss: 0.6934 - acc: 0.50 - ETA: 13s - loss: 0.6934 - acc: 0.50 - ETA: 13s - loss: 0.6931 - acc: 0.52 - ETA: 13s - loss: 0.6930 - acc: 0.54 - ETA: 13s - loss: 0.6930 - acc: 0.53 - ETA: 13s - loss: 0.6929 - acc: 0.55 - ETA: 13s - loss: 0.6929 - acc: 0.54 - ETA: 13s - loss: 0.6929 - acc: 0.54 - ETA: 12s - loss: 0.6930 - acc: 0.54 - ETA: 12s - loss: 0.6930 - acc: 0.54 - ETA: 12s - loss: 0.6930 - acc: 0.54 - ETA: 12

530/530 [==============================] - ETA: 15s - loss: 0.6940 - acc: 0.25 - ETA: 15s - loss: 0.6932 - acc: 0.50 - ETA: 15s - loss: 0.6932 - acc: 0.50 - ETA: 15s - loss: 0.6934 - acc: 0.43 - ETA: 15s - loss: 0.6936 - acc: 0.35 - ETA: 14s - loss: 0.6934 - acc: 0.41 - ETA: 14s - loss: 0.6934 - acc: 0.42 - ETA: 14s - loss: 0.6934 - acc: 0.43 - ETA: 14s - loss: 0.6933 - acc: 0.44 - ETA: 14s - loss: 0.6934 - acc: 0.42 - ETA: 14s - loss: 0.6934 - acc: 0.43 - ETA: 14s - loss: 0.6933 - acc: 0.43 - ETA: 14s - loss: 0.6933 - acc: 0.44 - ETA: 14s - loss: 0.6933 - acc: 0.46 - ETA: 13s - loss: 0.6933 - acc: 0.46 - ETA: 13s - loss: 0.6933 - acc: 0.46 - ETA: 13s - loss: 0.6932 - acc: 0.48 - ETA: 13s - loss: 0.6933 - acc: 0.45 - ETA: 13s - loss: 0.6932 - acc: 0.47 - ETA: 13s - loss: 0.6933 - acc: 0.46 - ETA: 13s - loss: 0.6933 - acc: 0.46 - ETA: 13s - loss: 0.6933 - acc: 0.46 - ETA: 13s - loss: 0.6932 - acc: 0.48 - ETA: 12s - loss: 0.6932 - acc: 0.50 - ETA: 12s - loss: 0.6932 - acc: 0.48 - ETA: 12

132/132 [==============================] - ETA: 2s - loss: 0.6931 - acc: 0.500 - ETA: 1s - loss: 0.6932 - acc: 0.484 - ETA: 0s - loss: 0.6932 - acc: 0.479 - ETA: 0s - loss: 0.6932 - acc: 0.476 - 1s 10ms/sample - loss: 0.6932 - acc: 0.4773
Epoch 1/5


...

530/530 [==============================] - ETA: 1:33 - loss: 0.6638 - acc: 0.750 - ETA: 52s - loss: 0.6631 - acc: 0.750 - ETA: 39s - loss: 0.6955 - acc: 0.66 - ETA: 32s - loss: 0.7539 - acc: 0.56 - ETA: 28s - loss: 0.7043 - acc: 0.65 - ETA: 25s - loss: 0.7473 - acc: 0.58 - ETA: 23s - loss: 0.7610 - acc: 0.53 - ETA: 21s - loss: 0.7467 - acc: 0.56 - ETA: 20s - loss: 0.7408 - acc: 0.55 - ETA: 19s - loss: 0.7355 - acc: 0.55 - ETA: 18s - loss: 0.7279 - acc: 0.56 - ETA: 17s - loss: 0.7251 - acc: 0.56 - ETA: 17s - loss: 0.7079 - acc: 0.59 - ETA: 16s - loss: 0.7106 - acc: 0.58 - ETA: 16s - loss: 0.6995 - acc: 0.60 - ETA: 15s - loss: 0.7351 - acc: 0.56 - ETA: 15s - loss: 0.7263 - acc: 0.57 - ETA: 14s - loss: 0.7188 - acc: 0.58 - ETA: 14s - loss: 0.7226 - acc: 0.56 - ETA: 14s - loss: 0.7277 - acc: 0.53 - ETA: 13s - loss: 0.7260 - acc: 0.53 - ETA: 13s - loss: 0.7233 - acc: 0.54 - ETA: 13s - loss: 0.7257 - acc: 0.53 - ETA: 13s - loss: 0.7280 - acc: 0.52 - ETA: 12s - loss: 0.7266 - acc: 0.52 - ETA:

530/530 [==============================] - ETA: 12s - loss: 0.6294 - acc: 0.75 - ETA: 11s - loss: 0.6876 - acc: 0.62 - ETA: 11s - loss: 0.7903 - acc: 0.41 - ETA: 11s - loss: 0.7410 - acc: 0.56 - ETA: 11s - loss: 0.7748 - acc: 0.50 - ETA: 11s - loss: 0.7675 - acc: 0.50 - ETA: 11s - loss: 0.7586 - acc: 0.50 - ETA: 11s - loss: 0.7527 - acc: 0.50 - ETA: 11s - loss: 0.7397 - acc: 0.52 - ETA: 11s - loss: 0.7420 - acc: 0.50 - ETA: 11s - loss: 0.7431 - acc: 0.47 - ETA: 11s - loss: 0.7422 - acc: 0.45 - ETA: 11s - loss: 0.7376 - acc: 0.46 - ETA: 11s - loss: 0.7326 - acc: 0.50 - ETA: 11s - loss: 0.7262 - acc: 0.53 - ETA: 11s - loss: 0.7241 - acc: 0.53 - ETA: 10s - loss: 0.7132 - acc: 0.55 - ETA: 10s - loss: 0.7126 - acc: 0.55 - ETA: 10s - loss: 0.7054 - acc: 0.56 - ETA: 10s - loss: 0.6909 - acc: 0.58 - ETA: 10s - loss: 0.6948 - acc: 0.58 - ETA: 10s - loss: 0.7253 - acc: 0.55 - ETA: 10s - loss: 0.7169 - acc: 0.56 - ETA: 10s - loss: 0.7135 - acc: 0.56 - ETA: 10s - loss: 0.7119 - acc: 0.57 - ETA: 10

530/530 [==============================] - ETA: 11s - loss: 0.6408 - acc: 0.75 - ETA: 12s - loss: 0.6599 - acc: 0.62 - ETA: 11s - loss: 0.6779 - acc: 0.50 - ETA: 11s - loss: 0.6638 - acc: 0.50 - ETA: 12s - loss: 0.6459 - acc: 0.55 - ETA: 12s - loss: 0.6637 - acc: 0.50 - ETA: 11s - loss: 0.6616 - acc: 0.53 - ETA: 11s - loss: 0.6456 - acc: 0.59 - ETA: 11s - loss: 0.6417 - acc: 0.61 - ETA: 11s - loss: 0.6297 - acc: 0.62 - ETA: 11s - loss: 0.6285 - acc: 0.63 - ETA: 11s - loss: 0.6186 - acc: 0.66 - ETA: 11s - loss: 0.6154 - acc: 0.67 - ETA: 11s - loss: 0.6032 - acc: 0.67 - ETA: 11s - loss: 0.6054 - acc: 0.66 - ETA: 11s - loss: 0.6180 - acc: 0.64 - ETA: 11s - loss: 0.6191 - acc: 0.63 - ETA: 10s - loss: 0.6218 - acc: 0.61 - ETA: 10s - loss: 0.6207 - acc: 0.61 - ETA: 10s - loss: 0.6157 - acc: 0.62 - ETA: 10s - loss: 0.6173 - acc: 0.61 - ETA: 10s - loss: 0.6195 - acc: 0.62 - ETA: 10s - loss: 0.6323 - acc: 0.61 - ETA: 10s - loss: 0.6211 - acc: 0.63 - ETA: 10s - loss: 0.6167 - acc: 0.64 - ETA: 10

530/530 [==============================] - ETA: 11s - loss: 0.4203 - acc: 0.75 - ETA: 11s - loss: 0.5313 - acc: 0.62 - ETA: 12s - loss: 0.6268 - acc: 0.50 - ETA: 11s - loss: 0.6188 - acc: 0.56 - ETA: 11s - loss: 0.5893 - acc: 0.65 - ETA: 11s - loss: 0.5604 - acc: 0.70 - ETA: 11s - loss: 0.5636 - acc: 0.71 - ETA: 11s - loss: 0.5502 - acc: 0.75 - ETA: 11s - loss: 0.5520 - acc: 0.75 - ETA: 11s - loss: 0.6025 - acc: 0.70 - ETA: 11s - loss: 0.6013 - acc: 0.70 - ETA: 11s - loss: 0.6127 - acc: 0.68 - ETA: 11s - loss: 0.5901 - acc: 0.71 - ETA: 11s - loss: 0.5780 - acc: 0.73 - ETA: 11s - loss: 0.5663 - acc: 0.73 - ETA: 11s - loss: 0.5690 - acc: 0.71 - ETA: 10s - loss: 0.5607 - acc: 0.73 - ETA: 10s - loss: 0.5442 - acc: 0.75 - ETA: 10s - loss: 0.5273 - acc: 0.76 - ETA: 10s - loss: 0.5196 - acc: 0.76 - ETA: 10s - loss: 0.5130 - acc: 0.76 - ETA: 10s - loss: 0.5238 - acc: 0.76 - ETA: 10s - loss: 0.5049 - acc: 0.77 - ETA: 10s - loss: 0.4896 - acc: 0.78 - ETA: 10s - loss: 0.4905 - acc: 0.78 - ETA: 10

132/132 [==============================] - ETA: 0s - loss: 0.4050 - acc: 0.812 - ETA: 0s - loss: 0.3622 - acc: 0.828 - ETA: 0s - loss: 0.3517 - acc: 0.854 - ETA: 0s - loss: 0.3604 - acc: 0.859 - 1s 4ms/sample - loss: 0.3806 - acc: 0.8485
Average Validation Score:  0.6893939


In [10]:
print('Average Validation Score: ', np.average(validation_scores))

Average Validation Score:  0.6893939
